In [570]:
! pip install rdflib Owlready2 -q

In [571]:
import rdflib

g = rdflib.Graph()

g.parse('simpsons.owl', format='xml')

<Graph identifier=N7250481d74ef4165a4df6607365df44d (<class 'rdflib.graph.Graph'>)>

In [572]:
from owlready2 import *

owlready2.JAVA_EXE = 'C:\\Program Files (x86)\\Java\\jdk-17\\bin\\java'

In [573]:
onto = get_ontology("simpsons.owl").load()

In [574]:
is_father_of = {
    'simpsons.AbrahamSimpson': [ind for ind in onto.individuals() if str(ind) in ['simpsons.HerbertPowell', 'simpsons.HomeroSimpson']],
    'simpsons.ClancyBouvier': [ind for ind in onto.individuals() if str(ind) in ['simpsons.MargeSimpson', 'simpsons.PattyBouvier', 'simpsons.SelmaBouvier']],
    'simpsons.HomeroSimpson': [ind for ind in onto.individuals() if str(ind) in ['simpsons.BartSimpson', 'simpsons.LisaSimpson', 'simpsons.MaggieSimpson']],
}

is_mother_of = {
    'simpsons.MonaSimpson': [ind for ind in onto.individuals() if str(ind) in ['simpsons.HomeroSimpson']],
    'simpsons.JacquelineBouvier': [ind for ind in onto.individuals() if str(ind) in ['simpsons.MargeSimpson', 'simpsons.PattyBouvier', 'simpsons.SelmaBouvier']],
    'simpsons.MargeSimpson': [ind for ind in onto.individuals() if str(ind) in ['simpsons.BartSimpson', 'simpsons.LisaSimpson', 'simpsons.MaggieSimpson']],
    'simpsons.SelmaBouvier': [ind for ind in onto.individuals() if str(ind) in ['simpsons.LingBouvier']],
}

## Ejercicio 2

Agregar relaciones entre padres e hijos

In [575]:
for person in onto.get_instances_of(onto.Hombre):
    if str(person) in is_father_of:
        sons = is_father_of[str(person)]
        person.es_padre_de = sons

In [576]:
for person in onto.get_instances_of(onto.Mujer):
    if str(person) in is_mother_of:
        sons = is_mother_of[str(person)]
        person.es_madre_de = sons

In [577]:
sync_reasoner_pellet(onto, infer_property_values=True, infer_data_property_values=True)

* Owlready2 * Running Pellet...
    C:\Program Files (x86)\Java\jdk-17\bin\java -Xmx2000M -cp C:\Python310\lib\site-packages\owlready2\pellet\antlr-3.2.jar;C:\Python310\lib\site-packages\owlready2\pellet\antlr-runtime-3.2.jar;C:\Python310\lib\site-packages\owlready2\pellet\aterm-java-1.6.jar;C:\Python310\lib\site-packages\owlready2\pellet\commons-codec-1.6.jar;C:\Python310\lib\site-packages\owlready2\pellet\httpclient-4.2.3.jar;C:\Python310\lib\site-packages\owlready2\pellet\httpcore-4.2.2.jar;C:\Python310\lib\site-packages\owlready2\pellet\jcl-over-slf4j-1.6.4.jar;C:\Python310\lib\site-packages\owlready2\pellet\jena-arq-2.10.0.jar;C:\Python310\lib\site-packages\owlready2\pellet\jena-core-2.10.0.jar;C:\Python310\lib\site-packages\owlready2\pellet\jena-iri-0.9.5.jar;C:\Python310\lib\site-packages\owlready2\pellet\jena-tdb-0.10.0.jar;C:\Python310\lib\site-packages\owlready2\pellet\jgrapht-jdk1.5.jar;C:\Python310\lib\site-packages\owlready2\pellet\log4j-1.2.16.jar;C:\Python310\lib\site-pa

In [578]:
for padre in onto.get_instances_of(onto.Padre):
    print('Hijos de {}:'.format(str(padre)), padre.es_padre_de)

for madre in onto.get_instances_of(onto.Madre):
    print('Hijos de {}:'.format(str(madre)), padre.es_padre_de) # Me sale vacío con es_madre_de ?????

Hijos de simpsons.AbrahamSimpson: [simpsons.HerbertPowell, simpsons.HomeroSimpson]
Hijos de simpsons.ClancyBouvier: [simpsons.MargeSimpson, simpsons.PattyBouvier, simpsons.SelmaBouvier]
Hijos de simpsons.HomeroSimpson: [simpsons.BartSimpson, simpsons.LisaSimpson, simpsons.MaggieSimpson]
Hijos de simpsons.JacquelineBouvier: [simpsons.BartSimpson, simpsons.LisaSimpson, simpsons.MaggieSimpson]
Hijos de simpsons.MargeSimpson: [simpsons.BartSimpson, simpsons.LisaSimpson, simpsons.MaggieSimpson]
Hijos de simpsons.MonaSimpson: [simpsons.BartSimpson, simpsons.LisaSimpson, simpsons.MaggieSimpson]
Hijos de simpsons.SelmaBouvier: [simpsons.BartSimpson, simpsons.LisaSimpson, simpsons.MaggieSimpson]


## Ejercicio 3

In [579]:
rule_abuelo_paterno = Imp(namespace=onto)
rule_abuela_paterna = Imp(namespace=onto)
rule_abuelo_materno = Imp(namespace=onto)
rule_abuela_materna = Imp(namespace=onto)

rule_nieto_paterno = Imp(namespace=onto)
rule_nieto_materno = Imp(namespace=onto)
rule_nieta_paterna = Imp(namespace=onto)
rule_nieta_materna = Imp(namespace=onto)

rule_abuelo_paterno.set_as_rule("""es_padre_de(?a, ?p) ^ es_padre_de(?p, ?h) -> es_abuelo_de(?a, ?h)""")
rule_abuela_paterna.set_as_rule("""es_madre_de(?a, ?p) ^ es_padre_de(?p, ?h) -> es_abuela_de(?a, ?h)""")
rule_abuelo_materno.set_as_rule("""es_padre_de(?a, ?p) ^ es_madre_de(?p, ?h) -> es_abuelo_de(?a, ?h)""")
rule_abuelo_materno.set_as_rule("""es_madre_de(?a, ?p) ^ es_madre_de(?p, ?h) -> es_abuela_de(?a, ?h)""")

rule_nieto_paterno.set_as_rule("""Hombre(?h) ^ es_abuelo_de(?a, ?h) -> es_nieto_de(?h, ?a)""")
rule_nieta_paterna.set_as_rule("""Mujer(?h) ^ es_abuelo_de(?a, ?h) -> es_nieta_de(?h, ?a)""")
rule_nieto_materno.set_as_rule("""Hombre(?h) ^ es_abuela_de(?a, ?h) -> es_nieto_de(?h, ?a)""")
rule_nieta_materna.set_as_rule("""Mujer(?h) ^ es_abuela_de(?a, ?h) -> es_nieta_de(?h, ?a)""")

Mujer(?h), es_abuela_de(?a, ?h) -> es_nieta_de(?h, ?a)

In [580]:
sync_reasoner_pellet(onto, infer_property_values=True, infer_data_property_values=True)

* Owlready2 * Running Pellet...
    C:\Program Files (x86)\Java\jdk-17\bin\java -Xmx2000M -cp C:\Python310\lib\site-packages\owlready2\pellet\antlr-3.2.jar;C:\Python310\lib\site-packages\owlready2\pellet\antlr-runtime-3.2.jar;C:\Python310\lib\site-packages\owlready2\pellet\aterm-java-1.6.jar;C:\Python310\lib\site-packages\owlready2\pellet\commons-codec-1.6.jar;C:\Python310\lib\site-packages\owlready2\pellet\httpclient-4.2.3.jar;C:\Python310\lib\site-packages\owlready2\pellet\httpcore-4.2.2.jar;C:\Python310\lib\site-packages\owlready2\pellet\jcl-over-slf4j-1.6.4.jar;C:\Python310\lib\site-packages\owlready2\pellet\jena-arq-2.10.0.jar;C:\Python310\lib\site-packages\owlready2\pellet\jena-core-2.10.0.jar;C:\Python310\lib\site-packages\owlready2\pellet\jena-iri-0.9.5.jar;C:\Python310\lib\site-packages\owlready2\pellet\jena-tdb-0.10.0.jar;C:\Python310\lib\site-packages\owlready2\pellet\jgrapht-jdk1.5.jar;C:\Python310\lib\site-packages\owlready2\pellet\log4j-1.2.16.jar;C:\Python310\lib\site-pa

In [581]:
for nieto in onto.get_instances_of(onto.Nieto):
    print('Abuelos de {}:'.format(str(nieto)), nieto.es_nieto_de)

for nieta in onto.get_instances_of(onto.Nieta):
    print('Abuelos de {}:'.format(str(nieta)), nieta.es_nieta_de)

Abuelos de simpsons.BartSimpson: [simpsons.MonaSimpson, simpsons.JacquelineBouvier, simpsons.AbrahamSimpson]
Abuelos de simpsons.LingBouvier: [simpsons.JacquelineBouvier]
Abuelos de simpsons.LisaSimpson: [simpsons.MonaSimpson, simpsons.JacquelineBouvier, simpsons.AbrahamSimpson]
Abuelos de simpsons.MaggieSimpson: [simpsons.MonaSimpson, simpsons.JacquelineBouvier, simpsons.AbrahamSimpson]


## Ejercicio 4

In [582]:
rule_bro_bro = Imp(namespace=onto)
rule_bro_sis = Imp(namespace=onto)
rule_sis_sis = Imp(namespace=onto)

rule_bro_bro.set_as_rule("""es_hijo_de(?a, ?p) ^ es_hijo_de(?b, ?p) ^ differentFrom(?a, ?b) -> es_hermano_de(?a, ?b)""")
rule_bro_sis.set_as_rule("""es_hijo_de(?a, ?p) ^ es_hija_de(?b, ?p) ^ differentFrom(?a, ?b) -> es_hermano_de(?a, ?b)""")
rule_sis_sis.set_as_rule("""es_hija_de(?a, ?p) ^ es_hija_de(?b, ?p) ^ differentFrom(?a, ?b) -> es_hermana_de(?a, ?b)""")

es_hija_de(?a, ?p), es_hijo_de(?b, ?p), DifferentFrom(?a, ?b) -> es_hermana_de(?a, ?b)

In [583]:
sync_reasoner_pellet(onto, infer_property_values=True, infer_data_property_values=True)

* Owlready2 * Running Pellet...
    C:\Program Files (x86)\Java\jdk-17\bin\java -Xmx2000M -cp C:\Python310\lib\site-packages\owlready2\pellet\antlr-3.2.jar;C:\Python310\lib\site-packages\owlready2\pellet\antlr-runtime-3.2.jar;C:\Python310\lib\site-packages\owlready2\pellet\aterm-java-1.6.jar;C:\Python310\lib\site-packages\owlready2\pellet\commons-codec-1.6.jar;C:\Python310\lib\site-packages\owlready2\pellet\httpclient-4.2.3.jar;C:\Python310\lib\site-packages\owlready2\pellet\httpcore-4.2.2.jar;C:\Python310\lib\site-packages\owlready2\pellet\jcl-over-slf4j-1.6.4.jar;C:\Python310\lib\site-packages\owlready2\pellet\jena-arq-2.10.0.jar;C:\Python310\lib\site-packages\owlready2\pellet\jena-core-2.10.0.jar;C:\Python310\lib\site-packages\owlready2\pellet\jena-iri-0.9.5.jar;C:\Python310\lib\site-packages\owlready2\pellet\jena-tdb-0.10.0.jar;C:\Python310\lib\site-packages\owlready2\pellet\jgrapht-jdk1.5.jar;C:\Python310\lib\site-packages\owlready2\pellet\log4j-1.2.16.jar;C:\Python310\lib\site-pa

In [586]:
for hermano in onto.get_instances_of(onto.Hermano):
    print('Hermanos de {}:'.format(str(hermano)), hermano.es_hermano_de)

for hermana in onto.get_instances_of(onto.Hermana):
    print('Hermanos de {}:'.format(str(hermana)), hermano.es_hermana_de)

Hermanos de simpsons.BartSimpson: [simpsons.MaggieSimpson, simpsons.LisaSimpson]
Hermanos de simpsons.HerbertPowell: [simpsons.HomeroSimpson]
Hermanos de simpsons.HomeroSimpson: [simpsons.HerbertPowell]
Hermanos de simpsons.LisaSimpson: []
Hermanos de simpsons.MaggieSimpson: []
Hermanos de simpsons.MargeSimpson: []
Hermanos de simpsons.PattyBouvier: []
Hermanos de simpsons.SelmaBouvier: []
